In [ ]:
# @title 0. Mount Google Drive (เชื่อมต่อไดรฟ์)
from google.colab import drive
import os

drive.mount('/content/drive')

print("✅ เชื่อมต่อเรียบร้อย!")

In [ ]:
import pandas as pd
import os
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# ==========================================
# 2. ตั้งค่าไฟล์
# ==========================================
# 🟢 ใส่ Path ของไฟล์ Temp 0
file_temp0_path = '/content/drive/MyDrive/ReaLearn/FY 2025/Interns - NU, Batch 3/AJCC_2026/Chanidapha/Result/qwen2.5-3b/temp0_Topp0.07/sentiment_vocab/VoteResult_qwen2.5_3b_temp0_sentiment_vocab.csv'

# 🟢 ใส่ Path ของไฟล์ Temp 1
file_temp1_path = '/content/drive/MyDrive/ReaLearn/FY 2025/Interns - NU, Batch 3/AJCC_2026/Chanidapha/Result/qwen2.5-3b/temp1_Topp0.07/sentiment_vocab/VoteResult_qwen2.5_3b_temp1_sentiment_temporal.csv'

# 🟢 ข้อมูลกำกับ
model_name = "qwen2.5-3b"
dataset_name = "sentiment_vocab"

output_folder = '/content/drive/MyDrive/ReaLearn/FY 2025/Interns - NU, Batch 3/AJCC_2026/Chanidapha/Result'
output_filename = 'Global_Analysis_Result.csv' 

col_vote = 'Vote'
col_answer = 'Expected_answer'

# ==========================================
# 3. Global Weighted Average
# ==========================================
def get_global_metrics(df, temp_label):
    # เตรียมข้อมูล
    y_true = df[col_answer].astype(str).str.strip()
    y_pred = df[col_vote].astype(str).str.strip()

    # 1. Accuracy
    acc = accuracy_score(y_true, y_pred)

    # 2. Precision, Recall, F1 
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    return [{
        'Model': model_name,
        'Dataset': dataset_name,
        'Temperature': temp_label,
        'Precision': round(prec, 4),
        'Recall': round(rec, 4),
        'F1-Score': round(f1, 4),
        'Accuracy': round(acc, 4)
    }]

# ==========================================
# 4. ประมวลผล
# ==========================================
all_results = []

try:
    # --- Temp 0 ---
    print(f"กำลังประมวลผล Temp 0...")
    df0 = pd.read_csv(file_temp0_path)
    if col_vote in df0.columns and col_answer in df0.columns:
        results_t0 = get_global_metrics(df0, "Temp 0")
        all_results.extend(results_t0)
    else:
        print(f"❌ Temp 0 ไม่พบคอลัมน์ที่กำหนด")

    # --- Temp 1 ---
    print(f"กำลังประมวลผล Temp 1...")
    df1 = pd.read_csv(file_temp1_path)
    if col_vote in df1.columns and col_answer in df1.columns:
        results_t1 = get_global_metrics(df1, "Temp 1")
        all_results.extend(results_t1)
    else:
        print(f"❌ Temp 1 ไม่พบคอลัมน์ที่กำหนด")

    # ==========================================
    # 5. บันทึกผล
    # ==========================================
    if all_results:
        df_final = pd.DataFrame(all_results)

        # จัดลำดับคอลัมน์
        cols = ['Model', 'Dataset', 'Temperature', 'Precision', 'Recall', 'F1-Score', 'Accuracy']
        df_final = df_final[cols]

        save_path = os.path.join(output_folder, output_filename)

        # เขียนต่อไฟล์เดิม (Append)
        if os.path.exists(save_path):
            df_final.to_csv(save_path, mode='a', header=False, index=False)
            print(f"\n✅ เพิ่มข้อมูลต่อท้ายไฟล์ {output_filename} เรียบร้อย!")
        else:
            df_final.to_csv(save_path, index=False)
            print(f"\n✅ สร้างไฟล์ใหม่ {output_filename} เรียบร้อย!")

        print(f"📂 ไฟล์อยู่ที่: {save_path}")
        print("-" * 60)
        # แสดงผลลัพธ์
        print(df_final.to_string(index=False))

    else:
        print("ไม่ข้อมูลให้บันทึก")

except Exception as e:
    print(f"Error: {e}")